In [1]:
import numpy as np

In [2]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [3]:
def numerical_derivative(f, x): # 수치미분 debug version # Ctrl + A 모두 선택
    delta_x = 1e-4 
    grad = np.zeros_like(x)
    
#     print("debug 1. initial input variable =", x)
#     print("debug 2. initial input grad =", grad)
#     print("================================== =")
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx=it.multi_index
        
#         print("debug 3, idx=",idx,",x[idx]=",x[idx])
        
        tmp_val = x[idx]
        x[idx]=float(tmp_val)+delta_x
        fx1=f(x) #f(x+delta_x)
        
        x[idx] = tmp_val - delta_x
        fx2 = f(x) # f(x-delta_x)
        grad[idx]=(fx1-fx2)/(2*delta_x)
        
#         print("debug 4. grad[idx] =",grad[idx])
#         print("debug 5. grad = ", grad)
#         print("=================================")
        
        x[idx] = tmp_val
        it.iternext() 
    return grad

In [4]:
#딥러닝으로 게이트 만들기
class LogicGate:
    def __init__(self,gate_name,xdata,tdata):
#         입력층 2개
        self.name=gate_name
        self.__xdata=xdata
        self.__tdata=tdata
        
        self.__xdata=xdata.reshape(4,2)
        self.__tdata=tdata.reshape(4,1)
#         은닉층 한층에 노드를 6개
        self.__w2=np.random.rand(2,6)
        self.__b2=np.random.rand(1)
#         출력층 한개
        self.__w3=np.random.rand(6,1)
        self.__b3=np.random.rand(1)
        
        self.__learning_rate=1e-2
        
# 입력층 은닉층 출력층
        
    def feed_forward(self):
#         입력층
        delta=1e-7
#         은닉층
        z2=np.dot(self.__xdata, self.__w2)+self.__b2
        a2=sigmoid(z2)
#         출력층
        z3=np.dot(a2, self.__w3)+self.__b3
        y=sigmoid(z3)
        
        return -np.sum(self.__tdata*np.log(y+delta)+(1-self.__tdata)*np.log((1-y)+delta))
    
#     트레이닝
    def train(self):
        f=lambda x:self.feed_forward()
        print("Initial loss func: ",self.feed_forward())
        
        for step in range(10001):
            self.__w2-=self.__learning_rate*numerical_derivative(f,self.__w2)
            self.__b2-=self.__learning_rate*numerical_derivative(f,self.__b2)
            
            self.__w3-=self.__learning_rate*numerical_derivative(f,self.__w3)
            self.__b3-=self.__learning_rate*numerical_derivative(f,self.__b3)
            
            if step %400==0:
                print("step: ",step,", loss_value: ",self.feed_forward())
                
    def predict(self, xdata):
        z2=np.dot(xdata, self.__w2)+ self.__b2
        a2=sigmoid(z2)
        
        z3=np.dot(a2, self.__w3)+ self.__b3
        y=sigmoid(z3)
        
        if y>=0.5:
            result=1
            
        else:
            result=0
            
        return y,result        

In [5]:
#and
xdata=np.array([[0,0],[0,1],[1,0],[1,1]])
tdata=np.array([0,0,0,1])

and_obj=LogicGate("AND_GATE",xdata,tdata)
and_obj.train()

#or
xdata=np.array([[0,0],[0,1],[1,0],[1,1]])
tdata=np.array([0,1,1,1])
or_obj=LogicGate("OR_GATE",xdata,tdata)
or_obj.train()

#nand
xdata=np.array([[0,0],[0,1],[1,0],[1,1]])
tdata=np.array([1,1,1,0])
nand_obj=LogicGate("NAND_GATE",xdata,tdata)
nand_obj.train()

#xor 
xdata=np.array([[0,0],[0,1],[1,0],[1,1]])
tdata=np.array([0,1,1,0])
xor_obj=LogicGate("XOR_GATE",xdata,tdata)
# XOR 게이트
xor_obj.train()

Initial loss func:  7.029834723249024
step:  0 , loss_value:  6.7548146694021804
step:  400 , loss_value:  2.2938924996753296
step:  800 , loss_value:  2.2128284877434434
step:  1200 , loss_value:  2.093426745375205
step:  1600 , loss_value:  1.882107322858039
step:  2000 , loss_value:  1.6441988520137802
step:  2400 , loss_value:  1.402160461011797
step:  2800 , loss_value:  1.1367985672547136
step:  3200 , loss_value:  0.8784755078791389
step:  3600 , loss_value:  0.6650037506336989
step:  4000 , loss_value:  0.5062291139014178
step:  4400 , loss_value:  0.39320429371766213
step:  4800 , loss_value:  0.3131030105089068
step:  5200 , loss_value:  0.25546495110953343
step:  5600 , loss_value:  0.21303486955818618
step:  6000 , loss_value:  0.1810316779801297
step:  6400 , loss_value:  0.15632697269294127
step:  6800 , loss_value:  0.1368498077075031
step:  7200 , loss_value:  0.12120268123212485
step:  7600 , loss_value:  0.10842190832003085
step:  8000 , loss_value:  0.097828366857021

In [7]:
# end
print(and_obj.name)
test_data=np.array([[0,0],[0,1],[1,0],[1,1]])

for input_data in test_data:
    print(and_obj.predict(input_data))
    (real_val, logical_val)=and_obj.predict(input_data)
    print(input_data,"=",logical_val)
print()
# or
print(or_obj.name)
test_data=np.array([[0,0],[0,1],[1,0],[1,1]])

for input_data in test_data:
    (real_val, logical_val)=or_obj.predict(input_data)
    print(input_data,"=",logical_val)
print()
    
# nand
print(nand_obj.name)
test_data=np.array([[0,0],[0,1],[1,0],[1,1]])

for input_data in test_data:
    (real_val, logical_val)=nand_obj.predict(input_data)
    print(input_data,"=",logical_val)
print()

# xor 익스큐트오아
print(xor_obj.name)
test_data=np.array([[0,0],[0,1],[1,0],[1,1]])

for input_data in test_data:
    (real_val, logical_val)=xor_obj.predict(input_data)
    print(input_data,"=",logical_val)

AND_GATE
(array([0.00214761]), 0)
[0 0] = 0
(array([0.01894615]), 0)
[0 1] = 0
(array([0.01888828]), 0)
[1 0] = 0
(array([0.97624794]), 1)
[1 1] = 1

OR_GATE
[0 0] = 0
[0 1] = 1
[1 0] = 1
[1 1] = 1

NAND_GATE
[0 0] = 1
[0 1] = 1
[1 0] = 1
[1 1] = 0

XOR_GATE
[0 0] = 0
[0 1] = 1
[1 0] = 1
[1 1] = 0
